In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
train_df = pd.read_csv('../preprocessing_data/preprocessed_train.csv').drop('Id', axis= 1)
test_df = pd.read_csv('../preprocessing_data/preprocessed_test.csv').drop('Id', axis= 1)
X = np.array(train_df.drop(['SalePrice'], axis = 1).values)
y = np.log1p(np.array(train_df['SalePrice'].values))
X_test = np.array(test_df.values)

In [3]:
from bayes_opt import BayesianOptimization
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import get_scorer, mean_squared_error
import time

In [4]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size= 0.2, random_state=42)

In [6]:
def lasso_cl_bo(alpha, max_iter):
    params_lasso = {}
    params_lasso['alpha'] = alpha
    params_lasso['max_iter'] = round(max_iter)
    
    lasso = Lasso(random_state= 123, **params_lasso)
    lasso.fit(X_train, y_train)

    score = -rmse(y_valid, lasso.predict(X_valid))
    return score

# Run Bayesian Optimization
start = time.time()
params_lasso ={
    'alpha':(0, 0.01),
    'max_iter': (1000, 3000),
}
lasso_bo = BayesianOptimization(lasso_cl_bo, params_lasso, random_state=111)
lasso_bo.maximize(init_points=20, n_iter=4)
print('It takes %s minutes' % ((time.time() - start)/60))

|   iter    |  target   |   alpha   | max_iter  |
-------------------------------------------------
| 1         | -0.1273   | 0.006122  | 1.338e+03 |
| 2         | -0.1248   | 0.004361  | 2.539e+03 |
| 3         | -0.1234   | 0.002953  | 1.298e+03 |
| 4         | -0.1204   | 0.0002248 | 1.84e+03  |
| 5         | -0.1223   | 0.002387  | 1.675e+03 |
| 6         | -0.1321   | 0.009907  | 1.475e+03 |
| 7         | -0.1178   | 0.0008119 | 2.339e+03 |
| 8         | -0.1275   | 0.006212  | 1.549e+03 |
| 9         | -0.1252   | 0.004662  | 1.237e+03 |
| 10        | -0.1179   | 0.0007396 | 2.802e+03 |
| 11        | -0.1296   | 0.00794   | 2.681e+03 |
| 12        | -0.1299   | 0.008152  | 2.982e+03 |
| 13        | -0.1268   | 0.005773  | 2.628e+03 |
| 14        | -0.1246   | 0.004213  | 1.055e+03 |
| 15        | -0.125    | 0.004541  | 1.211e+03 |
| 16        | -0.1299   | 0.008172  | 2.395e+03 |
| 17        | -0.1266   | 0.005653  | 1.548e+03 |
| 18        | -0.1322   | 0.009985  | 1.276e+03 |


In [9]:
lasso = Lasso(alpha = 0.0008119, max_iter= 2339)
lasso.fit(X_train, y_train)
rmse(y_valid, lasso.predict(X_valid))

0.11778937954176705

In [10]:
ans = pd.read_csv('../submission/cheat.csv').drop('Id', axis = 1)
ans = np.array(ans.values).reshape(1, -1)[0]
y_pred = lasso.predict(X_test)
print(f"Error: {rmse(np.log1p(ans), y_pred)}")

Error: 0.12630942885112117
